In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import os

import scanpy as sc
import seaborn as sns

from scroutines import basicu

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tools.sm_exceptions import ValueWarning
from tqdm import tqdm


import lmm

In [ ]:
outfigdir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_nrdr_lmm'
!mkdir $outfigdir

In [ ]:
sex_genes = [
    "Xist",
    "Uty",
    "Eif2s3y",
    "Kdm5d",
    "Ddx3y",
]

f1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw_multiome_l23.h5ad'
# f2 = '/u/home/f/f7xiesnm/v1_multiome/multiome_cell_sex_assignment_saumya.csv'

adata  = sc.read(f1)
# df_sex = pd.read_csv(f2)
# adata.obs = adata.obs.join(df_sex.set_index('cell'))
adata.X = adata.raw.X
adata = adata[:,~adata.var.index.str.contains(f'^mt')]
adata = adata[:,[g for g in adata.var.index if g not in sex_genes]]

genes = adata.var.index.values
adata

In [ ]:
%%time

time = 'P21'
exp_conds = [time, time+'DR']
subclass  = 'L2/3'
offset = 1e-2
scale = 1e4

adatasub = adata[(adata.obs['Age'].isin(exp_conds)) & (adata.obs['Subclass']==subclass)]

# ### test
# adatasub = adatasub[:,:20]
# genes = genes[:20]
# ### test

obs_fixed = 'Age'
obs_random = 'Sample'
obs = adatasub.obs[[obs_fixed, obs_random]].copy()
obs = obs.dropna()

adatasub = adatasub[obs.index]

output = os.path.join(outfigdir, f'NRDR_DEGs_LMM_{time}.csv')

# mat
mat = np.array(adatasub.X.todense())/adatasub.obs['total_counts'].values.reshape(-1,1)*scale

df_res = lmm.run_lmm(mat, genes, obs, obs_fixed, obs_random, output=output, offset=offset)

In [ ]:
df_res

In [ ]:
df_res.set_index('gene').loc['Nptx2']

In [ ]:
df_res.set_index('gene').loc['Igsf9b']